<a href="https://colab.research.google.com/github/addinar/toxicity-detector/blob/main/backend/model/toxicity_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Login to HuggingFace**

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `my-token-2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `my-toke

# **Load Data**

In [ ]:
!pip install datasets

Access the dataset [here](https://huggingface.co/datasets/Koushim/processed-jigsaw-toxic-comments).

In [ ]:
from datasets import load_dataset

ds = load_dataset("Koushim/processed-jigsaw-toxic-comments")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/34.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/97320 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/97320 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = ds['train'].to_pandas()
test = ds['test'].to_pandas()
validation = ds['validation'].to_pandas()

In [ ]:
train.head()

,text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,labels,input_ids,token_type_ids,attention_mask
0,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[101, 2023, 2003, 2061, 4658, 1012, 2009, 1005...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[101, 4067, 2017, 999, 999, 2023, 2052, 2191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[101, 2023, 2003, 2107, 2019, 13661, 2640, 329...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[101, 2003, 2023, 2242, 1045, 1005, 2222, 2022...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.0,0.87234,0.021277,0.0,"[0.8936170339584351, 0.021276595070958138, 0.0...","[101, 5292, 3270, 2017, 4364, 2024, 1037, 9129...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


Since we already have `input_ids` and `attention_mask`, we can skip the tokenizing step.

In [ ]:
labels = train.columns[1:8]
labels

Index(['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult',
       'identity_attack', 'sexual_explicit'],
      dtype='object')

# **Data Preparation**

In [ ]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
import sys
sys.path.append('/content')

Make sure to have `model/models/dataset.py` and `model/models/regressor.py`loaded in notebook files.

In [ ]:
from dataset import Dataset

train_dataset = Dataset(train)
val_dataset = Dataset(validation)
test_dataset = Dataset(test)

In [ ]:
from torch.utils.data import Subset

train_dataset = Subset(train_dataset, range(0, 20000))
val_dataset = Subset(val_dataset, range(0, 5000))
test_dataset = Subset(test_dataset, range(0, 5000))

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# **Training Loop (MSELoss)**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from regressor import Regressor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Regressor().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
len(train_loader)

1250

In [ ]:
for epoch in range(1): # 1 epoch
  model.train()
  total_loss = 0
  for batch_idx, batch in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

    if batch_idx % 1000 == 0:
      print(f'[Batch {batch_idx}] Loss: {loss.item():.4f}')

  print(f'Epoch {epoch+1} Loss: {total_loss:.4f}')

  0%|          | 1/1250 [00:10<3:46:34, 10.88s/it]

[Batch 0] Loss: 0.0883


 80%|████████  | 1001/1250 [2:40:26<39:29,  9.51s/it]

[Batch 1000] Loss: 0.0046


100%|██████████| 1250/1250 [3:20:24<00:00,  9.62s/it]

Epoch 1 Loss: 8.8710


# **Testing Loop**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
len(test_loader)

157

In [ ]:
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
  for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids, attention_mask)
    all_preds.append(outputs.cpu().numpy())
    all_targets.append(labels.cpu().numpy())

predicted_scores = np.concatenate(all_preds, axis=0)
true_scores = np.concatenate(all_targets, axis=0)

mse = mean_squared_error(true_scores, predicted_scores)
mae = mean_absolute_error(true_scores, predicted_scores)
r2 = r2_score(true_scores, predicted_scores)

print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R² Score: {r2:.4f}")

MSE: 0.0059
MAE: 0.0422
R² Score: 0.2588


In [ ]:
true_argmax = np.argmax(true_scores, axis=1)
pred_argmax = np.argmax(predicted_scores, axis=1)

In [ ]:
accuracy = np.mean(true_argmax == pred_argmax)
print(f"Argmax Match Accuracy: {accuracy:.4f}")

Argmax Match Accuracy: 0.7384


In [ ]:
y_pred_binary = (predicted_scores > 0.5).astype(int)
y_true_binary = (true_scores > 0.5).astype(int)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print("Micro F1:", f1_score(y_true_binary, y_pred_binary, average='micro'))
print("Macro F1:", f1_score(y_true_binary, y_pred_binary, average='macro'))
print("Sample-wise Accuracy:", accuracy_score(y_true_binary, y_pred_binary))

Micro F1: 0.6302186878727635
Macro F1: 0.2403794357789442
Sample-wise Accuracy: 0.9526


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **Test**

Before I retrain, I will check if the toxicity detector works with comments I give.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
comment = "You're the dumbest person I've ever met. Just crawl back under whatever rock you came from, nobody wants you here." # generated by Gemini

In [ ]:
encoding = tokenizer(comment, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

In [ ]:
model.eval()
with torch.no_grad():
    output = model(input_ids, attention_mask)
    prediction = output.cpu().numpy()[0]  # shape: (7,)

In [ ]:
for label, score in zip(labels, prediction):
    print(f"{label}: {score:.4f}")

toxicity: 0.8055
severe_toxicity: 0.0684
obscene: 0.0742
threat: 0.0499
insult: 0.7868
identity_attack: -0.0851
sexual_explicit: -0.0257


In [ ]:
torch.save(model.state_dict(), "model_v2.pt")